In [17]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Creamos las funciones para el Modelo de Recomendacion

## Abrimos la tabla procesada anteriormente

- random_merged_recommend_model.parquet

In [18]:
random_merged_recommend_content = '../FastAPI/merged_recommend_model.parquet'
random_merged_recommend_model = pd.read_parquet(random_merged_recommend_content)

## Funciones

### A continuación probamos diferentes funciones para usar cosine_similarity y llevarla a FastAPI 

In [19]:
# Filtrar los datos relevantes (recommend=True y sentimientos válidos)
relevant_data = random_merged_recommend_model[(random_merged_recommend_model['recommend'] == True) & 
                          random_merged_recommend_model['sentiment_analysis'].isin([0, 1, 2])]

# Construir la matriz de ítem-ítem usando pivot_table
item_item_matrix = relevant_data.pivot_table(index='app_name', columns='item_id', values='sentiment_analysis', fill_value=0)

# Calcular la similitud de coseno entre ítems
item_similarity = cosine_similarity(item_item_matrix.T)

# Función para obtener ítems similares a un ítem dado por su nombre ('app_name')
def get_similar_items(item_name, num_similar_items=5):
    item_id = item_item_matrix.index.get_loc(item_name)
    similar_indices = item_similarity[item_id].argsort()[::-1][1:num_similar_items+1]
    similar_items = []
    for index in similar_indices:
        similar_items.append(item_item_matrix.index[index])
    return similar_items

# Obtener recomendaciones para el ítem 'Commander Keen'
recommended_items = get_similar_items('Commander Keen', num_similar_items=5)

# Mostrar los ítems más similares al ítem 'Commander Keen'
print(recommended_items)

['eversion', 'Frontlines™: Fuel of War™', 'Feeding Frenzy 2 Deluxe', 'Final DOOM', 'FlatOut']


In [20]:
# Filtrar los datos relevantes (recommend=True y sentimientos válidos)
relevant_data = random_merged_recommend_model[(random_merged_recommend_model['recommend'] == True) & 
                          random_merged_recommend_model['sentiment_analysis'].isin([0, 1, 2])]

# Construir la matriz de ítem-ítem usando pivot_table
item_item_matrix = relevant_data.pivot_table(index='item_id', columns='app_name', values='sentiment_analysis', fill_value=0)

# Calcular la similitud de coseno entre ítems
item_similarity = cosine_similarity(item_item_matrix.T)

# Función para obtener ítems similares a un ítem dado por su id ('item_id')
def get_similar_items(item_name, num_similar_items=5):
    item_id = item_item_matrix.index.get_loc(item_name)
    similar_indices = item_similarity[item_id].argsort()[::-1][1:num_similar_items+1]
    similar_items = []
    for index in similar_indices:
        similar_items.append(item_item_matrix.index[index])
    return similar_items

# Obtener recomendaciones para el ítem '20'
recommended_items = get_similar_items(20, num_similar_items=5)
print(recommended_items)

filtered_names = random_merged_recommend_model[random_merged_recommend_model['item_id'].isin(recommended_items)]
selected_names = filtered_names[['item_id', 'app_name']]
unique_names = selected_names.drop_duplicates(subset='item_id').reset_index(drop=True)
print('\n',unique_names)

[55230, 9450, 8930, 8980, 9010]

    item_id                                     app_name
0     8930                  Sid Meier's Civilization® V
1     8980                                  Borderlands
2     9010                 Return to Castle Wolfenstein
3     9450  Warhammer® 40,000: Dawn of War® - Soulstorm
4    55230                        Saints Row: The Third


In [21]:
# Filtrar datos relevantes
relevant_data = random_merged_recommend_model[(random_merged_recommend_model['recommend'] == True) & 
                                              random_merged_recommend_model['sentiment_analysis'].isin([0, 1, 2])]

# Construir matriz de usuario-ítem
user_item_matrix = relevant_data.pivot_table(index='user_id', columns='item_id', values='sentiment_analysis', fill_value=0)

# Calcular similitud entre usuarios
user_similarity = cosine_similarity(user_item_matrix)

def get_user_recommendations(user_id, num_recommendations=5):
    # Obtener la fila correspondiente al usuario
    user_row = user_item_matrix.loc[user_id].values.reshape(1, -1)
    # Calcular similitud entre el usuario y otros usuarios
    similarity_scores = cosine_similarity(user_row, user_item_matrix.values)
    # Obtener los usuarios más similares (excluyendo al propio usuario)
    similar_users_indices = similarity_scores.argsort()[0][-num_recommendations-1:-1]
    # Obtener los ítems que los usuarios similares han recomendado
    recommended_items = user_item_matrix.iloc[similar_users_indices].sum().sort_values(ascending=False).index.tolist()
    return recommended_items

# Ejemplo: Obtener recomendaciones para el usuario con user_id 'thesnozberry'
user_id = 'thesnozberry'

recommendations = get_user_recommendations(user_id)
top_5_recommendations = recommendations[:5]
print(top_5_recommendations)

filtered_names = random_merged_recommend_model[random_merged_recommend_model['item_id'].isin(top_5_recommendations)]
selected_names = filtered_names[['item_id', 'app_name']]
unique_names = selected_names.drop_duplicates(subset='item_id').reset_index(drop=True)
print('\n',unique_names)

[400, 32370, 10, 24870, 25890]

    item_id                                   app_name
0       10                             Counter-Strike
1      400                                     Portal
2    24870                      Need for Speed: Shift
3    25890                         Hearts of Iron III
4    32370  STAR WARS™ - Knights of the Old Republic™


In [22]:
def recomendacion_juego(user_id: str, num_recommendations=5):
    relevant_data = random_merged_recommend_model[(random_merged_recommend_model['recommend'] == True) &
                                                  random_merged_recommend_model['sentiment_analysis'].isin([0, 1, 2])]
    user_item_matrix = relevant_data.pivot_table(
        index='user_id', columns='item_id', values='sentiment_analysis', fill_value=0)
    user_similarity = cosine_similarity(user_item_matrix)
    user_row = user_item_matrix.loc[user_id].values.reshape(1, -1)
    similarity_scores = cosine_similarity(user_row, user_item_matrix.values)
    similar_users_indices = similarity_scores.argsort()[
        0][-num_recommendations-1:-1]
    recommended_items = user_item_matrix.iloc[similar_users_indices].sum(
    ).sort_values(ascending=False).index.tolist()

    return recommended_items


user_id = 'thesnozberry'

recommendations = recomendacion_juego(user_id)
top_5_recommendations = recommendations[:5]
print(top_5_recommendations)

[400, 32370, 10, 24870, 25890]


In [23]:
def recomendacion_juego(item_id: int) -> list:
    num_similar_items = 5
    item_id = item_item_matrix.index.get_loc(item_id)
    similar_indices = item_similarity[item_id].argsort()[
        ::-1][1:num_similar_items+1]
    similar_items = [int(item_item_matrix.index[index])
                     for index in similar_indices]

    game_names = [
        f"Juego recomendado {i+1}: {random_merged_recommend_model.loc[
            random_merged_recommend_model['item_id'] == item_id,
            'app_name'].values[0]}" for i,
        item_id in enumerate(similar_items)]

    return game_names
print(recomendacion_juego(55110))


['Juego recomendado 1: Saints Row: The Third', 'Juego recomendado 2: Supreme Commander: Forged Alliance', 'Juego recomendado 3: Freedom Force', "Juego recomendado 4: Sid Meier's Civilization® V", 'Juego recomendado 5: Borderlands']


In [24]:
def recomendacion_usuario2(user_id: str) -> list:
    user_row = user_item_matrix.loc[user_id].values.reshape(1, -1)
    similarity_scores = cosine_similarity(user_row, user_item_matrix.values)
    similar_users_indices = similarity_scores.argsort()[0][-6:-1]
    recommended_items = user_item_matrix.iloc[similar_users_indices].sum(
    ).sort_values(ascending=False).index.tolist()[:5]

    game_names = [
        f"Juego recomendado {
            i+1}: {random_merged_recommend_model.loc[random_merged_recommend_model['item_id'] == item_id, 'app_name'].values[0]}"
        for i, item_id in enumerate(recommended_items)
    ]

    return game_names
print(recomendacion_usuario2('ruemz1'))

['Juego recomendado 1: Counter-Strike: Source', 'Juego recomendado 2: Counter-Strike', 'Juego recomendado 3: Medal of Honor: Airborne', "Juego recomendado 4: King's Bounty: The Legend", 'Juego recomendado 5: Hearts of Iron III']
